> To live! like a tree alone and free,
> and like a forest in solidarity...
> Nazim Hikmet

<center> <h1> Random Forests (Yes! No Forest Image) </h1> </center>



__Objectives__

- Introduction of 'bagging' procedure.

- Identifying the need for bootstrapping for random forests

- Comparing Random forests and bagging methods

- Evaluating a model by random forest model

# Review: Bootstrapping


- Boostrapping is a resampling technique

<img src= "img/bootstrap1.png" style="height:250px">


# Bagging: Bootstrap Aggregation


__Core idea__

- Decision trees have high variance.

- We know averaging can reduce the variance. (Central Limit Theorem?)

- Why bootstrap though?

    - Averaging the same numbers yields the same number: [2,2,2,2]  --> avg: 2
    
    - You want each tree to learn a little bit differently so that averaging make sense.

__A little bit more details__


We will use this idea calculate $$ \hat{f}^{1}(x), \cdots, \hat{f}^{B}(x)$$ where each $\hat{f}^{i}$ represents a decision tree fitted to the bootstrapped data.

Then we will make a prediction by: 

$$ \hat{f}_{\text{avg}}(x) = \frac{1}{B}\sum_{b=1}^{B} \hat{f}^{b}(x)$$

Note that this is for regression and for the classification we can get majority vote.

[sklearn averages over probabilities not majority vote](https://scikit-learn.org/stable/modules/ensemble.html#forest)


## Random Forests

__Problem__ We still have some problem with this approach and random forests will address this problem. Can you see the issue?

- If we have a strong predictor then this will dominate in each tree.

Hint: Correlated trees

- Random forests adds some randomness into the procedure to make each tree uncorrolated.

## Sklearn for RandomForests

In [1]:
import pandas as pd

In [2]:
## you can download the data from -- https://www.kaggle.com/ishaanv/ISLR-Auto#Heart.csv

## or http://faculty.marshall.usc.edu/gareth-james/ISL/data.html
heart = pd.read_csv('data/Heart.csv', index_col = 0)
heart.head()
print(heart.shape)

(303, 14)


In [3]:
heart.dropna(axis= 0, how= 'any', inplace = True)
y = heart.AHD
heart.drop(columns= 'AHD', inplace = True)

In [4]:
heart.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal
1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed
2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal
3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable
4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal
5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal


In [5]:
heart.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297 entries, 1 to 302
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        297 non-null    int64  
 1   Sex        297 non-null    int64  
 2   ChestPain  297 non-null    object 
 3   RestBP     297 non-null    int64  
 4   Chol       297 non-null    int64  
 5   Fbs        297 non-null    int64  
 6   RestECG    297 non-null    int64  
 7   MaxHR      297 non-null    int64  
 8   ExAng      297 non-null    int64  
 9   Oldpeak    297 non-null    float64
 10  Slope      297 non-null    int64  
 11  Ca         297 non-null    float64
 12  Thal       297 non-null    object 
dtypes: float64(2), int64(9), object(2)
memory usage: 32.5+ KB


In [6]:
display(heart.Thal.value_counts())
display(heart.ChestPain.value_counts())

normal        164
reversable    115
fixed          18
Name: Thal, dtype: int64

asymptomatic    142
nonanginal       83
nontypical       49
typical          23
Name: ChestPain, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split 
X_train, X_test,y_train, y_test = train_test_split(heart, y, test_size= 0.20, stratify = y)

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:
categorical_variables = X_train.select_dtypes(include=['object']).columns
numerical_variables = X_train.select_dtypes(include = ['int64', 'float64']).columns

In [10]:
import numpy as np

In [11]:
ohe = OneHotEncoder(drop = 'first')
X_categ = ohe.fit_transform(X_train[categorical_variables]).toarray()
X_num = X_train[numerical_variables].values
Xtrain = np.concatenate((X_categ, X_num), axis = -1,)
Xtrain.shape

(237, 16)

In [12]:
## now we should transform the test data 
## to be able to use it for the prediction

X_test_categ = ohe.transform(X_test[categorical_variables]).toarray()
X_test_num = X_test[numerical_variables].values
Xtest = np.concatenate((X_test_categ, X_test_num), axis = -1,)
Xtest.shape

(60, 16)

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [44]:
clf = RandomForestClassifier(n_estimators= 1000, 
                             criterion= 'gini', 
                             max_features= 'sqrt',
                             max_depth= 4,
                             min_samples_leaf= 6,
                             oob_score= True,
                             n_jobs = -1)

__n_estimators:__  number of trees to train
__criterion:__ For each tree splitting criteria
__max_features:__ How to choose number of features that an individual tree will be given during its training procedure.
__oob_scores__: 


__Your Turn__

- Use 5 fold cross_validation to fit random forest classifier we created above.
- Don't forget to return training scores and trained estimators.

In [45]:
clf.fit(Xtrain, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=4, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [29]:
clf.score(Xtrain, y_train)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 300 out of 300 | elapsed:    0.0s finished


0.8987341772151899

In [16]:
from sklearn.model_selection import cross_validate

In [41]:
# %load -r 1-6 supplement.py
cv = cross_validate(estimator = clf,
                    X = Xtrain,
                    y = y_train,
                    return_train_score=True,
                    return_estimator=True,
                    cv=5)

In [46]:
clf.oob_score_

0.8227848101265823

In [42]:
 cv['test_score']

array([0.83333333, 0.875     , 0.82978723, 0.80851064, 0.78723404])

In [43]:
 cv['train_score']

array([0.87301587, 0.87830688, 0.89473684, 0.88421053, 0.87894737])

In [ ]:
c

__Your Turn__

- What is the type of validator above?

- Check test vs train(validation) scores.

- Print "mean +/- std" for both train and test scores

- Also print oob_scores and compare them with cross_validation scores

In [17]:
# %load -r 9-17 supplement.py
mean_test = np.mean(cv['test_score'])
std_test = np.std(cv['test_score'])

mean_train = np.mean(cv['train_score'])
std_train = np.std(cv['train_score'])


print("Train score: %.3f +/- %.3f"%(mean_train, std_train))
print("Test score: %.3f +/- %.3f"%(mean_test, std_test))

Train score: 1.000 +/- 0.000
Test score: 0.768 +/- 0.050


In [18]:
clf.fit(Xtrain, y_train)
clf.oob_score_

0.7763713080168776

__Your Turn__

- Note that we have over-fitting problem. 

- Let's try to reduce over-fitting

In [19]:
y_train.value_counts()

No     128
Yes    109
Name: AHD, dtype: int64

In [20]:
# %load -r 20-27 supplement.py
clf = RandomForestClassifier(n_estimators= 300,
                             criterion= 'gini',
                             max_features= 'auto', max_depth= 7,
                             oob_score= True)

clf.fit(Xtrain, y_train);
clf.score(Xtrain, y_train)
clf.oob_score_

0.7721518987341772

### Extra: Pipelines?

In [21]:
## There is an "easier" way to do this
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [22]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop='first'))])

preprocessor = ColumnTransformer(
        transformers=[
        ('num', numeric_transformer, numerical_variables),
        ('cat', categorical_transformer, categorical_variables)],)

rf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators =100,
                                                            oob_score = True))])

In [23]:
pipe_validator = cross_validate(rf, 
                    X_train, 
                    y_train, 
                    return_train_score= True, 
                    return_estimator= True,
                    cv = 5)

In [24]:
## train scores
print(cv['train_score'])
## validation scores
print(cv['test_score'])

## let's pick one of the estimator for further investigation

est = cv['estimator'][0]

[1. 1. 1. 1. 1.]
[0.79166667 0.8125     0.68085106 0.80851064 0.74468085]


## Continue...

In [61]:
feature_importances = clf.feature_importances_

In [62]:
feature_importances

array([0.04346569, 0.02221601, 0.01686114, 0.11053086, 0.06494023,
       0.09076995, 0.02765893, 0.06729764, 0.06547854, 0.00834202,
       0.01619441, 0.134579  , 0.07879943, 0.10948582, 0.04939825,
       0.09398206])

In [63]:
# be careful with the order of columns
columns = ohe.get_feature_names().tolist() +numerical_variables.tolist()

In [64]:
importances = pd.DataFrame(data= feature_importances, index = columns, columns= ['feature_importances'])

importances.sort_values(by = 'feature_importances', ascending = False)

,feature_importances
MaxHR,0.134579
x1_normal,0.110531
Oldpeak,0.109486
Ca,0.093982
Age,0.090770
ExAng,0.078799
RestBP,0.067298
Chol,0.065479
x1_reversable,0.064940
Slope,0.049398


### Extra Material 

- [Sklearn averages probabilities in RF implementation](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [On the variance](https://newonlinecourses.science.psu.edu/stat414/node/167/)

- [Do RF immune to overfitting?](https://en.wikipedia.org/wiki/Talk%3ARandom_forest)

- [Tricky stuff with respect to feature importance](http://rnowling.github.io/machine/learning/2015/08/10/random-forest-bias.html)

- [An interesting implementation of feature importance](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances_faces.html#sphx-glr-auto-examples-ensemble-plot-forest-importances-faces-py)

- [Different Ensemble Methods in sklearn](https://scikit-learn.org/stable/modules/ensemble.html#forest)

- [ISLR - section 8.2](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf)

- [Another library for RF: H2o](http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/drf.html)